In [1]:
file = open("Cthulhu.txt", mode='r',encoding="utf-8")
text = file.read()
n = len(text)
num_words = len(set(text))
print(f"我收集的Lovecraft小說共有 {n} 中文字")
print(f"包含了 {num_words} 個獨一無二的字")

我收集的Lovecraft小說共有 357538 中文字
包含了 3363 個獨一無二的字


In [2]:
import tensorflow as tf

# 初始化一個以字為單位的 Tokenizer
tokenizer = tf.keras\
    .preprocessing\
    .text\
    .Tokenizer(
        num_words=num_words,
        char_level=True,
        filters=''
)

# 讓 tokenizer 讀過全文，
# 將每個新出現的字加入字典並將中文字轉
# 成對應的數字索引
tokenizer.fit_on_texts(text)
text_as_int = tokenizer\
    .texts_to_sequences([text])[0]

# # 隨機選取一個片段文本方便之後做說明
# s_idx = 21004
# e_idx = 21020
# partial_indices = \
#     text_as_int[s_idx:e_idx]
# partial_texts = [
#     tokenizer.index_word[idx] \
#     for idx in partial_indices
# ]

# # 渲染結果，可忽略
# print("原本的中文字序列：")
# print()
# print(partial_texts)
# print()
# print("-" * 20)
# print()
# print("轉換後的索引序列：")
# print()
# print(partial_indices)

In [3]:
# _type = type(text_as_int)
# n = len(text_as_int)
# print(f"text_as_int 是一個 {_type}\n")
# print(f"小說的序列長度： {n}\n")
# print("前 5 索引：", text_as_int[:5])

In [4]:
# print("實際丟給模型的數字序列：")
# print(partial_indices[:-1])
# print()
# print("方便我們理解的文本序列：")
# print(partial_texts[:-1])

In [5]:
# 方便說明，實際上我們會用更大的值來
# 讓模型從更長的序列預測下個中文字
SEQ_LENGTH = 10  # 數字序列長度
BATCH_SIZE = 128 # 幾筆成對輸入/輸出

# text_as_int 是一個 python list
# 我們利用 from_tensor_slices 將其
# 轉變成 TensorFlow 最愛的 Tensor <3
characters = tf\
    .data\
    .Dataset\
    .from_tensor_slices(
        text_as_int)

# 將被以數字序列表示的文本
# 拆成多個長度為 SEQ_LENGTH (10) 的序列
# 並將最後長度不滿 SEQ_LENGTH 的序列捨去
sequences = characters\
    .batch(SEQ_LENGTH + 1, 
           drop_remainder=True)

#全文所包含的成對輸入/輸出的數量
steps_per_epoch = \
    len(text_as_int) // SEQ_LENGTH

# 這個函式專門負責把一個序列
# 拆成兩個序列，分別代表輸入與輸出
# （下段有 vis 解釋這在做什麼）
def build_seq_pairs(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# 將每個從文本擷取出來的序列套用上面
# 定義的函式，拆成兩個數字序列
# 作為輸入／輸出序列
# 再將得到的所有數據隨機打亂順序
# 最後再一次拿出 BATCH_SIZE（128）筆數據
# 作為模型一次訓練步驟的所使用的資料
ds = sequences\
    .map(build_seq_pairs)\
    .shuffle(steps_per_epoch)\
    .batch(BATCH_SIZE, 
           drop_remainder=True)

In [6]:
# 超參數
EMBEDDING_DIM = 512
RNN_UNITS = 1024

# 使用 keras 建立一個非常簡單的 LSTM 模型
model = tf.keras.Sequential()

# 詞嵌入層
# 將每個索引數字對應到一個高維空間的向量
model.add(
    tf.keras.layers.Embedding(
        input_dim=num_words, 
        output_dim=EMBEDDING_DIM,
        batch_input_shape=[
            BATCH_SIZE, None]
))

# LSTM 層
# 負責將序列數據依序讀入並做處理
model.add(
    tf.keras.layers.LSTM(
    units=RNN_UNITS, 
    return_sequences=True, 
    stateful=True, 
    recurrent_initializer='glorot_uniform'
))

# 全連接層
# 負責 model 每個中文字出現的可能性
model.add(
    tf.keras.layers.Dense(
        num_words))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 512)          1721856   
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         6295552   
_________________________________________________________________
dense (Dense)                (128, None, 3363)         3447075   
Total params: 11,464,483
Trainable params: 11,464,483
Non-trainable params: 0
_________________________________________________________________


In [7]:
# 超參數，決定模型一次要更新的步伐有多大
LEARNING_RATE = 0.001

# 定義模型預測結果跟正確解答之間的差異
# 因為全連接層沒使用 activation func
# from_logits= True 
def loss(y_true, y_pred):
    return tf.keras.losses\
    .sparse_categorical_crossentropy(
        y_true, y_pred, from_logits=True)

# 編譯模型，使用 Adam Optimizer 來最小化
# 剛剛定義的損失函數
model.compile(
    optimizer=tf.keras\
        .optimizers.Adam(
        learning_rate=LEARNING_RATE), 
    loss=loss
)


In [8]:
# EPOCHS = 10 # 決定看幾篇文本
# history = model.fit(
#     ds, # 前面使用 tf.data 建構的資料集
#     epochs=EPOCHS
# )

In [9]:
EPOCHS = 75
callbacks = [
    tf.keras.callbacks\
        .TensorBoard("logs"),
    # 你可以加入其他 callbacks 如
    # ModelCheckpoint,
    # EarlyStopping
]

history = model.fit(
    ds,
    epochs=EPOCHS, 
    callbacks=callbacks
)
model.save("model.h5")

In [10]:
%load_ext tensorboard
%tensorboard --logdir logs


Reusing TensorBoard on port 6006 (pid 14236), started 20:40:10 ago. (Use '!kill 14236' to kill it.)

In [11]:
# 跟訓練時一樣的超參數，
# 只差在 BATCH_SIZE 為 1
EMBEDDING_DIM = 512
RNN_UNITS = 1024
BATCH_SIZE = 1

# 專門用來做生成的模型
infer_model = tf.keras.Sequential()

# 詞嵌入層
infer_model.add(
    tf.keras.layers.Embedding(
        input_dim=num_words, 
        output_dim=EMBEDDING_DIM,
        batch_input_shape=[
            BATCH_SIZE, None]
))

# LSTM 層
infer_model.add(
    tf.keras.layers.LSTM(
    units=RNN_UNITS, 
    return_sequences=True, 
    stateful=True
))

# 全連接層
infer_model.add(
    tf.keras.layers.Dense(
        num_words))

# 讀入之前訓練時儲存下來的權重
infer_model.load_weights("model.h5")
infer_model.build(
    tf.TensorShape([1, None]))

In [12]:
seed_indices = [104, 349, 395, 294, 57, 16, 4, 82, 1, 776, 6, 487, 390, 22, 297, 241]

# 增加 batch 維度丟入模型取得預測結果後
# 再度降維，拿掉 batch 維度
input = tf.expand_dims(
    seed_indices, axis=0)
predictions = infer_model(input)
predictions = tf.squeeze(
    predictions, 0)

# 利用生成溫度影響抽樣結果
temperature = 0.7
predictions /= temperature

# 從 4330 個分類值中做抽樣
# 取得這個時間點模型生成的中文字
sampled_indices = tf.random\
    .categorical(
        predictions, 1)

In [13]:
start_seed = ['最', '終', '結', '果', '都', '是', '一', '樣', '的', '：', '在', '混', '亂', '到', '達', '頂']
input_eval = [tokenizer.word_index[s]\
                  for s in start_seed]
input_eval = tf.expand_dims(input_eval, axis=0)
print(input_eval)
pridection_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
pridection_id = tf.expand_dims([pridection_id], axis=0)
pridection_id = tf.cast(pridection_id,tf.int32)
print(pridection_id)
print(tf.concat([input_eval,pridection_id],1))

tf.Tensor([[104 349 395 294  57  16   4  82   1 776   6 487 390  22 297 241]], shape=(1, 16), dtype=int32)
tf.Tensor([[208]], shape=(1, 1), dtype=int32)
tf.Tensor([[104 349 395 294  57  16   4  82   1 776   6 487 390  22 297 241 208]], shape=(1, 17), dtype=int32)


In [14]:
def generate_text(model,start_seed,gen_size=20,temp=0.8):
    input_eval = [tokenizer.word_index[s]\
                  for s in start_seed]
    input_eval = tf.expand_dims(input_eval, axis=0)
    model.reset_states()
    for i in range(gen_size):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions /= temp
        pridection_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        pridection_id = tf.expand_dims([pridection_id], axis=0)
        pridection_id = tf.cast(pridection_id,tf.int32)
        input_eval = tf.concat([input_eval,pridection_id],1)
    output = [tokenizer.index_word[idx]\
                  for idx in input_eval.numpy().flatten()]
    return ''.join(output)
gen_text = generate_text(infer_model,"但最令他印象深刻的還是由密集的尖頂與高塔所組成的那副讓人不知所措的景象。這些尖塔聳立在遙遠的平原上。",150,0.5)
print(gen_text)

但最令他印象深刻的還是由密集的尖頂與高塔所組成的那副讓人不知所措的景象。這些尖塔聳立在遙遠的平原上。事後，只有一陣陣回音還在哀怨地念叨著那些長著威脅著外部世界北方大陸的海岸線，至於他們到底坦白了一樣，將之前的建築風格與要冒險穿過更多的街道，而且這條狗一定會在它們的幫助下找到了吉爾曼。他們已經意識到在修築這些東西的大部分皮膚都鮮明的展示在了二副那些生活在山巔之上、更加幽暗的街道上，而興奮的商人們進行
